In [ ]:
import json
import requests
from bs4 import BeautifulSoup

### Imports

2. Sending HTTP request to acces first page with opinions

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews_scroll"
response = requests.get(url)

3. Checking the code of http response

In [ ]:
response.status_code

4. Parse the html code of first page wth opinions

In [ ]:
page_dom = BeautifulSoup(response.text, 'html.parser')

5. Extract required data from the page

In [ ]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
for opinion in opinions:
    opinion_id = opinion["data-entry-id"]
    author = opinion.select_one("span.user-post__author-name").get_text()
    try:
        recommendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
    except AttributeError:
        recommendation = None
    stars = opinion.select_one("span.user-post__score-count").get_text()
    content = opinion.select_one("div.user-post__text").get_text()
    pros = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
    cons = [s.get_text() for s in opinion.select("div.review-feature__item--negative")]
    vote_yes = opinion.select_one("button.vote-yes")["data-total-vote"]
    vote_no = opinion.select_one("button.vote-no")["data-total-vote"]
    published = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
    try:
        purchased = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    except TypeError:
        purchased = None
    print(opinion_id, author, recommendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep = "\n")

6. If there are more pages, move to the next page and repeat steps 2-6 for each page

In [ ]:
try:
    next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
except TypeError:
    next_page = None

In [ ]:
next_page = "https://www.ceneo.pl/84514582#tab=reviews_scroll"
allOpinions = []
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            singleOpinion = {}
            singleOpinion["opinion_id"] = opinion["data-entry-id"]
            singleOpinion["author"] = opinion.select_one("span.user-post__author-name").get_text()
            try:
                singleOpinion["recommendation"] = opinion.select_one("span.user-post__author-recomendation > em").get_text()
            except AttributeError:
                singleOpinion["recommendation"] = None
            singleOpinion["stars"] = opinion.select_one("span.user-post__score-count").get_text()
            singleOpinion["content"] = opinion.select_one("div.user-post__text").get_text()
            singleOpinion["pros"] = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
            singleOpinion["cons"] = [s.get_text() for s in opinion.select("div.review-feature__item--negative")]
            singleOpinion["vote_yes"] = opinion.select_one("button.vote-yes")["data-total-vote"]
            singleOpinion["vote_no"] = opinion.select_one("button.vote-no")["data-total-vote"]
            singleOpinion["published"] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
            try:
                singleOpinion["purchased"] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
            except TypeError:
                singleOpinion["purchased"] = None
            allOpinions.append(singleOpinion)
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None
    else:
        print("Smth went wrong")
print(allOpinions)